## Check python version and tensorflow are all good

In [2]:
!python -V
!pyenv versions

Python 3.6.8
/bin/sh: 1: pyenv: not found


In [2]:
# !apt-get install wget
# !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-keyring_1.0-1_all.deb
# !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys F60F4B3D7FA2AF80
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys A4B469963BF863CC

# Use command on terminal
# !add-apt-repository -y ppa:jblgf0/python
# !add-apt-repository --remove ppa:jblgf0/python

Executing: /tmp/tmp.wo6WkVMRQV/gpg.1.sh --keyserver
keyserver.ubuntu.com
--recv-keys
A4B469963BF863CC
gpg: requesting key 3BF863CC from hkp server keyserver.ubuntu.com
gpg: key 3BF863CC: public key "cudatools <cudatools@nvidia.com>" imported
gpg: Total number processed: 1
gpg:               imported: 1  (RSA: 1)


In [3]:
!apt update -y
!apt install build-essential zlib1g-dev libncurses5-dev libgdbm-dev libnss3-dev libssl-dev libreadline-dev libffi-dev wget -y
!cd /root
!wget https://www.python.org/ftp/python/3.7.16/Python-3.7.16.tgz
# from IPython.display import clear_output 
# clear_output()
# !update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 2
# !update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.5 1
# !sudo update-alternatives --config python3

Get:1 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  InRelease
Ign:1 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  InRelease
Get:2 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  Release [574 B]
Get:2 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  Release [574 B]0m  
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  InRelease [1581 B]
Get:5 http://security.ubuntu.com/ubuntu xenial-security InRelease [99.8 kB]m   
Hit:6 http://archive.ubuntu.com/ubuntu xenial InRelease 
Get:7 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [99.8 kB]      
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Packages [518 kB]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release [564 B]
Get:11 https://developer.download.nvidia.com/compute/machine-learning/r

7Progress: [ 18%] [###########...............................................] 87Progress: [ 19%] [############..............................................] 8Selecting previously unselected package libtinfo-dev:amd64.
Preparing to unpack .../libtinfo-dev_6.0+20160213-1ubuntu1_amd64.deb ...
7Progress: [ 20%] [#############.............................................] 8Unpacking libtinfo-dev:amd64 (6.0+20160213-1ubuntu1) ...
7Progress: [ 22%] [#############.............................................] 87Progress: [ 23%] [##############............................................] 8Selecting previously unselected package libncurses5-dev:amd64.
Preparing to unpack .../libncurses5-dev_6.0+20160213-1ubuntu1_amd64.deb ...
7Progress: [ 24%] [###############...........................................] 8Unpacking libncurses5-dev:amd64 (6.0+20160213-1ubuntu1) ...
7Progress: [ 25%] [###############...........................................] 87Progress: [ 26%] [################.

7Progress: [ 76%] [#############################################.............] 87Progress: [ 77%] [##############################################............] 8Setting up libnspr4-dev (2:4.13.1-0ubuntu0.16.04.1) ...
7Progress: [ 79%] [##############################################............] 87Progress: [ 80%] [###############################################...........] 8Setting up libreadline6-dev:amd64 (6.3-8ubuntu2) ...
7Progress: [ 81%] [################################################..........] 87Progress: [ 82%] [################################################..........] 8Setting up libreadline-dev:amd64 (6.3-8ubuntu2) ...
7Progress: [ 83%] [#################################################.........] 87Progress: [ 84%] [##################################################........] 8Setting up libssl-dev:amd64 (1.0.2g-1ubuntu4.20) ...
7Progress: [ 86%] [##################################################........] 87Progress: [ 87%] [############################

In [2]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())

1.15.0
/device:GPU:0


In [2]:
import pandas as pd
import numpy as np
import pysbd # Sentence segmenter
import time, random, os, sys

In [6]:
# Import mongodb_forRead.py & PredictModel.py & secret.py
from modules.mongodb_forRead import MongoDB
import secret

# Database connection
mongoDB_connection = MongoDB()
currTime = int(time.time())
today = time.localtime(currTime)
today_to_timestamp = time.mktime((today.tm_year, today.tm_mon, today.tm_mday,8,0,0,0,0,0)) # mktime -> GTC+0 timestamp
prevDayNews = mongoDB_connection.read_prev24hrs_news(today_to_timestamp)
if not prevDayNews: 
    print("There is no esg-related news today!")
    sys.exit()

# ESG News sentence segmenter
esgNews = pd.Series([],dtype=pd.StringDtype()) 
seg = pysbd.Segmenter(language="en", clean=False)
for eachNew in prevDayNews:
    newSentences = seg.segment(eachNew)
    esgNews = pd.concat([esgNews, pd.Series([sentence.replace('\xa0',' ') for sentence in newSentences])], ignore_index=True)

# Write to TSV file
date = str(today.tm_year) + str(today.tm_mon) + str(today.tm_mday)
esgNews.to_csv("./esgBERT_input/esgNews_{}.tsv".format(date), sep='\t', header=False)
print("esgBERT input is ready!")

There is no esg-related news today!


SystemExit: 

c:\Users\me\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
class PredictModel:
    def __init__(self):
        self.config_path = "./predict_folder_config.json"
        with open('./import_data/label.json') as f:
            label_dict = json.load(f)
        # self.model_list = ['Carbon Emissions', 'Board', 'Privacy and Data Security', 'Supply Chain Labor Standards']
        self.model_list = list(label_dict.values())[:-1]
        self.msci_esg = {'Environment': ['Biodiversity and Land Use', 'Carbon Emissions', 'Climate Change Vulnerability', 'Electronic Waste', 'Financing Environmental Impact', 'Opportunities in Clean Tech', 'Opportunities in Green Building', 'Opportunities in Renewable Energy', 'Packaging Material and Waste', 'Product Carbon Footprint', 'Raw Material Sourcing', 'Toxic Emissions and Waste', 'Water Stress'], 'Social': ['Access to Communications', 'Access to Finance', 'Access to Health Care', 'Chemical Safety', 'Community Relations', 'Controversial Sourcing', 'Financial Product Safety', 'Health and Demographic Risk', 'Health and Safety', 'Human Capital Development', 'Labor Management', 'Opportunities in Nutrition and Health', 'Privacy and Data Security', 'Product Safety and Quality', 'Responsible Investment', 'Supply Chain Labor Standards'], 'Governance' : ['Accounting', 'Board', 'Business Ethics', 'Ownership and Control', 'Pay', 'Tax Transparency'] }
        with open(self.config_path) as f:
            self.config_dict = json.load(f)

    def get_config(self):
        with open(self.config_path) as f:
            config_dict = json.load(f)
        return config_dict
    
    def set_config(self, config_dict):
        with open(self.config_path, 'w') as f:
            json.dump(config_dict,f)
        print("Set Config Success")
        
    # Sometime the bert script will crash for no reason, and it will work after running again
    def run_model(self,file_name):
        fail_list = []
        for model in self.model_list:
            if not path.exists("{}/{}".format(self.config_dict['output_dir'],model)):
                print("Not exist")
                os.system('mkdir \"{}/{}\"'.format(self.config_dict["output_dir"],model))
            bert_script = ["python", '{}'.format(self.config_dict["script_name"]), '--task_name=class_2', '--do_predict=True', 
                           '\"--data_dir={}/{}\"'.format(self.config_dict["data_dir"],file_name), '\"--vocab_file={}/vocab.txt\"'.format(self.config_dict["pretrain_model_dir"]),
                           '\"--bert_config_file={}/bert_config.json\"'.format(self.config_dict["pretrain_model_dir"]),
                           '\"--init_checkpoint={}/{}/{}\"'.format(self.config_dict["finetune_mode_dir"],model,self.config_dict["checkpoint"]),
                           '--max_seq_length={}'.format(self.config_dict["max_seq_length"]),
                           '\"--output_dir={}/{}\"'.format(self.config_dict["output_dir"],model),
                           '\"--output_tsv_name={}\"'.format(file_name)
                          ]
            print(' '.join(bert_script))
            if os.system(' '.join(bert_script)) != 0:
                fail_list.append(model)

        if len(fail_list) == 0:
            return 'All model predict Success'
        else:
            return "{} fail!".format('/'.join(fail_list))
    
    def combine_predict_result(self, file_name):
        report_df = pd.DataFrame()
        for key_issue in self.model_list:
            tmp_df = pd.read_csv('{}/{}/{}'.format(self.config_dict["output_dir"],key_issue,file_name),sep='\t', names=['No',key_issue])
            if report_df.empty:
                report_df = tmp_df[key_issue].to_frame(name=key_issue)
            else:
                report_df[key_issue] = tmp_df[key_issue].values

        report_df.to_csv('{}/{}_table.csv'.format(self.config_dict["output_table_dir"], file_name[:-4])) 

In [ ]:
from modules.PredictModel import PredictModel
model = PredictModel()
model.get_config()

In [ ]:
currTime = int(time.time())
today = time.localtime(currTime)
today = str(today.tm_year) + str(today.tm_mon) + str(today.tm_mday)
# predict_msg = model.run_model("esgNews_{}.tsv".format(today))
predict_msg = model.run_model("esgNews_20221220.tsv".format(today))
print(predict_msg)

In [46]:
# result_1_df = pd.read_csv("./esgBERT_output/esgNews_{}_table.csv".format(today), index_col=0)
result_1_df = pd.read_csv("./esgBERT_output/esgNews_20221220_table.csv", index_col=0)
result_1_df['max'] = result_1_df.max(axis=1)
filter_df = result_1_df[result_1_df['max'] >= 0.9].iloc[:,:-1]
#沒有加columns就會是全部的index
one_hot_df = pd.DataFrame(np.where(filter_df.T == filter_df.T.max(), 1, 0),index=filter_df.columns).T
#one_hot_df = pd.DataFrame(np.where(filter_df.T == filter_df.T.max(), 1, 0),index=filter_df.columns, columns = filter_df.index).T
all_df = one_hot_df.sum().to_frame(name="Counts")
#count_df.index.name = "Key Issue"
# all_df = all_df.T
all_df.index.name = "Key Issues"
display(all_df)

,Counts
Key Issues,
Biodiversity and Land Use,33
Carbon Emissions,115
Climate Change Vulnerability,0
Electronic Waste,0
Financing Environmental Impact,0
Opportunities in Clean Tech,0
Opportunities in Green Building,3
Opportunities in Renewable Energy,30
Packaging Material and Waste,1


In [45]:
one_hot_df.loc[one_hot_df[all_df.idxmax().values[0]] == 1].index

Biodiversity and Land Use                 33
Carbon Emissions                         115
Climate Change Vulnerability               0
Electronic Waste                           0
Financing Environmental Impact             0
Opportunities in Clean Tech                0
Opportunities in Green Building            3
Opportunities in Renewable Energy         30
Packaging Material and Waste               1
Product Carbon Footprint                   7
Raw Material Sourcing                      0
Toxic Emissions and Waste                  0
Water Stress                              31
Accounting                                 0
Board                                    124
Business Ethics                          450
Ownership and Control                      0
Pay                                        0
Tax Transparency                           2
Access to Communications                   0
Access to Finance                          0
Access to Health Care                     51
Chemical S

In [9]:
one_hot_df

,Biodiversity and Land Use,Carbon Emissions,Climate Change Vulnerability,Electronic Waste,Financing Environmental Impact,Opportunities in Clean Tech,Opportunities in Green Building,Opportunities in Renewable Energy,Packaging Material and Waste,Product Carbon Footprint,...,Financial Product Safety,Health and Demographic Risk,Health and Safety,Human Capital Development,Labor Management,Opportunities in Nutrition and Health,Privacy and Data Security,Product Safety and Quality,Responsible Investment,Supply Chain Labor Standards
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1376,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1377,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
import pandas as pd
import numpy as np
esgNews = pd.read_csv("./esgBERT_input/esgNews_20221220.tsv", sep='\t', names=['idx', 'sentence'], index_col=0)
esgNews.iloc[2]["sentence"]

'“We went through six years of Stingers in 10 months,” Gregory J. Hayes, Raytheon’s chief executive, said in an interview earlier this month, referring to 1,600 of the company’s shoulder-fired antiaircraft missiles sent by the U.S. government to Ukraine. '

In [42]:
most_ky_sentences = [esgNews.iloc[idx]["sentence"] for idx in one_hot_df.loc[one_hot_df[all_df.idxmax().values[0]] == 1].index]
most_ky_sentences

['; Middletown, Iowa; and Kingsport, Tenn., where contractors work with the Army to manufacture the ammunition that Ukrainian artillery crews have burned through at an alarming rate. ',
 '(The money for these programs is expected to be included in a huge appropriations bill that appears to be on track to pass Congress and signed into law by Mr. Biden by the end of the week.)Spending could be even higher, as Congress is also considering a request for an extra $21.7 billion for the Pentagon, above the already expanded 2023 annual budget, to allocate more money to resupply materials used in Ukraine.In an indication of how government policy is shifting to rebuild industrial capacity for the military, Congress this year has moved to allow the Defense Department to more broadly make multiyear spending commitments for certain weapons systems and shipbuilding operations. ',
 'That is a provision that industry lobbyists have long pushed for, arguing it gives companies certainty that investments

In [59]:
with open("./outputfile", "w") as opfile:
    opfile.write("\n".join(most_ky_sentences))

In [61]:
import random
most_ky_sentences = random.sample(most_ky_sentences,30)

In [ ]:
new_df = pd.DataFrame(np.where(filter_df.T == filter_df.T.max(), 1, 0),index=filter_df.columns).T
sentence_categories = {} # {'Key Issue': ['sentence']}
sentences_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ESG_Step4/ESGNews.tsv', header=None, sep='\t')
for column in new_df.columns:
    sentence_categories[column] = [sentences_df.loc[ind][1] for ind, value in enumerate(new_df.loc[:, column]) if value == 1]

keyIssues_df = pd.DataFrame.from_dict(sentence_categories, orient='index')
keyIssues_df.insert(0, 'count', [len(sentence_categories[column]) for column in new_df.columns])
display(keyIssues_df)

keyIssues_df.to_csv("ESGNews-sentences-sorted.csv")

In [ ]:
tmp = keyIssues_df.T
tmp = tmp[:50]
tmp.to_csv("ESGNews-50-sentences-sorted.csv")

In [ ]:
pd.set_option('display.max_colwidth', None)
test = keyIssues_df.T
test

In [51]:
esgNews = pd.Series([],dtype=pd.StringDtype(), name='sentence')
newSentences = ['a','b','c']
esgNews = pd.concat([esgNews, pd.Series([sentence.replace('\xa0',' ') for sentence in newSentences])], ignore_index=True)
esgNews

0    a
1    b
2    c
dtype: object

## GPT generation

In [7]:
import os
import openai
from modules.secret import setup_api_key

setup_api_key() # adds api key to env
openai.organization = "org-hpKR7cri3YnOlgPZqIAZXtZv"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [66]:
prompt_input = '\n'.join(most_ky_sentences) + "\nWrite about 100 words tweet to summarize the news above (IMPORTANT: under 150 characters limit):\n"
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt_input,
  temperature=0.7,
  max_tokens=200,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

In [67]:
tweet = response['choices'][0].text
print(tweet)


CNN's Jeff Licht is making changes, from killing the network's streaming service to focusing on CNN.com. He's also bringing Republicans back to the network, while pushing for a "rational conversation about polarizing issues." #CNN #JeffLicht #RationalConversation


In [68]:
len(tweet)

264

## Twitter Test

In [16]:
import numpy as np
import tweepy

#variables for accessing twitter API
consumer_key='VVRuSF8zMLk7ERaCSc4ulZ4iM'
consumer_secret='KlYaRsIRH1HxBUjNq24sUts09w08dtu3QT6iioR2sWrat441PE'
access_token = "1602909982223400960-lFst1DZAYe9DWTL5NLxqCcj3oaWSXY"
access_token_secret = "gDhK48DtsCRY6xAocdHQp77cjWjcyXthRZOPZhYHph40W"

# OAuth 2.0 Bearer Token (App-Only)
# bearer_token = 'AAAAAAAAAAAAAAAAAAAAAMCxkQEAAAAA4cdJtTzMqDXuEul4uLV3NpFiQZs%3D3sI96h49nRftDbpYg43Hrg0mEj4F4RDQQrj4NAPNaq8L0MpE3b'

# Oauth2.0 Client ID & Sercet
# ClientID = 'bGI2ZE94SGpSNE9sOHdlY19IN0o6MTpjaQ'
# Client_sercet = '1MW-luKC73asmNkWkf1AxxS8zTZsO9PKwmmfiVzlD-e2yixGY8'

In [69]:
client = tweepy.Client(
    consumer_key=consumer_key, consumer_secret=consumer_secret,
    access_token=access_token, access_token_secret=access_token_secret
)

# Replace the text with whatever you want to Tweet about
response = client.create_tweet(text=tweet[:280])

print(response)

Response(data={'id': '1605839441594970112', 'text': 'CNN\'s Jeff Licht is making changes, from killing the network\'s streaming service to focusing on https://t.co/E0WDZZvBsR. He\'s also bringing Republicans back to the network, while pushing for a "rational conversation about polarizing issues." #CNN #JeffLicht #RationalConversation'}, includes={}, errors=[], meta={})
